In [1]:
import math

In [2]:
class Node:
    def __init__(self, val, parent = None):
        self.leftChild = None
        self.rightChild = None
        self.parent = parent
        self.value = val
        self.leftCount = 0
        self.rightCount = 0
        self.count = 1
        self.balance = 0
        
    def isLeftChild(self):
        return (not self.parent is None) and self.parent.leftChild is self
    
    def isRightChild(self):
        return (not self.parent is None) and self.parent.rightChild is self    

In [3]:
class Tree:
    def __init__(self):
        self.root = None
        self.numEntries = 0
        
    def add(self, val):
        self.numEntries += 1
        if self.root == None:
            self.root = Node(val)
        else:
            self._add(val, self.root)
    
    def _add(self, val, node):
        if val < node.value:
            node.leftCount += 1
            if node.leftChild is None:
                node.leftChild = Node(val, node)
                self.updateBalance(node.leftChild)
            else:                
                self._add(val, node.leftChild)
        elif val > node.value:
            node.rightCount += 1
            if node.rightChild is None:
                node.rightChild = Node(val, node)
                self.updateBalance(node.rightChild)
            else:                
                self._add(val, node.rightChild) 
        elif val == node.value:
            node.count +=1
    
    def updateBalance (self, node):
        if node.balance >1 or node.balance <-1:
            self.rebalance(node)
            return
        if not node.parent is None:
            if node.isLeftChild():
                node.parent.balance += 1
            elif node.isRightChild():
                node.parent.balance -=1    
            if node.parent.balance !=0:
                self.updateBalance(node.parent)
                
    def rotateLeft (self, oldRoot):
        newRoot = oldRoot.rightChild
        oldRoot.rightChild = newRoot.leftChild
        if not newRoot.leftChild is None:
            newRoot.leftChild.parent = oldRoot
        newRoot.parent = oldRoot.parent
        if oldRoot is self.root:
            self.root = newRoot
        else:
            if oldRoot.isLeftChild():
                oldRoot.parent.leftChild = newRoot
            else:
                oldRoot.parent.rightChild = newRoot
        newRoot.leftChild = oldRoot
        oldRoot.parent = newRoot
        #Update balance factor of nodes
        oldRoot.balance = oldRoot.balance + 1 - min (newRoot.balance, 0)
        newRoot.balance = newRoot.balance + 1 + max (oldRoot.balance, 0)
        #Update leftCount and rightCount of nodes
        oldRoot.rightCount = newRoot.leftCount
        newRoot.leftCount = oldRoot.rightCount + oldRoot.leftCount + oldRoot.count
        
    def rotateRight (self, oldRoot):
        newRoot = oldRoot.leftChild
        oldRoot.leftChild = newRoot.rightChild
        if not newRoot.rightChild is None:
            newRoot.rightChild.parent = oldRoot
        newRoot.parent = oldRoot.parent
        if oldRoot is self.root:
            self.root = newRoot
        else:
            if oldRoot.isRightChild():
                oldRoot.parent.rightChild = newRoot
            else:
                oldRoot.parent.leftChild = newRoot
        newRoot.rightChild = oldRoot
        oldRoot.parent = newRoot
        #Update balance factors of nodes
        oldRoot.balance = oldRoot.balance - 1 - max (newRoot.balance, 0)
        newRoot.balance = newRoot.balance - 1 + min (oldRoot.balance, 0)
        #Update leftCount and rightCount of nodes
        oldRoot.leftCount = newRoot.rightCount
        newRoot.rightCount = oldRoot.leftCount + oldRoot.rightCount +oldRoot.count
        
    def rebalance (self, node):
        if node.balance < 0:
            if node.rightChild.balance > 0:
                self.rotateRight (node.rightChild)
                self.rotateLeft (node)
            else:
                self.rotateLeft(node)
        elif node.balance > 0:
            if node.leftChild.balance < 0:
                self.rotateLeft (node.leftChild)
                self.rotateRight(node)
            else:
                self.rotateRight(node)
                
    def printTree (self):
        if not self.root is None:
            self._printTree(self.root)
    
    def _printTree (self, node):
        if not node is None:
            self._printTree(node.leftChild)
            print ('Value =', node.value, '. Left Count =', node.leftCount, '. Right Count = ', node.rightCount, '. Count = ', node.count)
            self._printTree (node.rightChild)
            
    def findRank(self, rank):
        if not self.root is None:
            return self._findRank(rank, self.root)
        else:
            return None
    
    def _findRank(self, rank, node):
        if node.leftChild is None:
            leftCount = 0
        else:
            leftCount = node.leftCount
        
        if rank >leftCount and rank <= leftCount + node.count:
            return node.value
        elif rank <= leftCount:
            if node.leftChild is None:
                return None
            return self._findRank(rank, node.leftChild)
        elif rank > leftCount + node.count:
            if node.rightChild is None:
                return None
            return self._findRank (rank-leftCount-node.count, node.rightChild)
        else:
            return None
    
    def findPercentile (self, percentile):
        #Find nearest-rank, given percentile
        
        #Special case: 0th percentile is rank 1. 
        if percentile == 0:
            return self.findRank(1)
        
        rank = math.ceil(percentile/100*self.numEntries)
        return self.findRank(rank)
        
        

            

In [4]:
math.ceil(45.334/100*1000)

454

In [35]:
import numpy as np

import time

def createTree (values):
    output = Tree()
    for value in values:
        output.add(value)
    return output
        
def createArray (values):
    return np.array(values)

def findArrayPercentile(array, percentile):
    if array is None or len(array)<=0:
        return None
    else:
        return np.percentile(array, math.ceil(percentile/100*len(array))/len(array)*100, interpolation = 'lower')

def testPercentile(values, percentile):
    start = time.time()
    tree = createTree(values)
    end = time.time()
    treeCreateTime = end - start
    
    start = time.time()
    array = createArray(values)
    end = time.time()
    arrayCreateTime = end - start
    
    start = time.time()
    treePercentile = tree.findPercentile(percentile)
    end = time.time()
    treePercentileTime = (end - start)
    
    start = time.time()
    arrayPercentile = findArrayPercentile(array, percentile)
    end = time.time()
    arrayPercentileTime = (end - start)
    
    print ('Values = ', values)
    print ('Percentile = ', percentile)
    print ('Tree Creation time=', treeCreateTime, '. Array Creation Time =', arrayCreateTime, 
           '. Tree Better?', treeCreateTime <= arrayCreateTime)
    print ('Tree Percentile time=', treePercentileTime, '. Array Percentile Time =', arrayPercentileTime, 
           '. Tree Better?', treePercentileTime <= arrayPercentileTime)
    print ('Tree percentile = ', treePercentile, '. Array percentile = ', arrayPercentile)
    print ()
    if treePercentile == arrayPercentile:
        print ('Success!')
        print ('')
    else:
        raise ValueError('Tree did not calculate percentile correctly')
        
def testPercentile2(values, percentile):
    treeCreateTime = 0
    treePercentileTime = 0
    arrayCreateTime = 0
    arrayPercentileTime = 0
    
    tree = Tree()
    array = np.array([])
    
    for value in values:
        start = time.time()
        tree.add(value)
        end = time.time()
        treeCreateTime += end - start
        
        start = time.time()
        treePercentile = tree.findPercentile(percentile)
        end = time.time()
        treePercentileTime += end - start
        
        start = time.time()
        array = np.append (array,value)
        end = time.time()
        arrayCreateTime += end - start
        
        start = time.time()
        arrayPercentile = findArrayPercentile(array, percentile)
        end = time.time()
        arrayPercentileTime += end - start
        
        if treePercentile != arrayPercentile:
            print ('Tree percentile = ', treePercentile, '. Array percentile = ', arrayPercentile)
            print ('Percentile = ', percentile)
            print ('Value = ', value)
            print ('Values = ', values)
            print ('Numpy array = ', array)

            raise ValueError('Tree did not calculate percentile correctly')
    

    print ('Tree Creation time=', treeCreateTime, '. Array Creation Time =', arrayCreateTime, 
           '. Tree Better?', treeCreateTime < arrayCreateTime)
    print ('Tree Percentile time=', treePercentileTime, '. Array Percentile Time =', arrayPercentileTime, 
           '. Tree Better?', treePercentileTime < arrayPercentileTime)
    print ()

        
        

In [38]:
#Test 4: Large array
import random

random.seed(1)

testValues = []

for i in range (0, 10000):
    testValues.append(random.randint(1,1000))

testPercentile2(testValues,5)
    

Tree Creation time= 0.04265737533569336 . Array Creation Time = 0.08651018142700195 . Tree Better? True
Tree Percentile time= 0.06089639663696289 . Array Percentile Time = 0.6339783668518066 . Tree Better? True



In [39]:
#Test 5: Repeated numbers

testValues = []

for i in range (0,5):
    testValues = []
    testValue = random.randint(-100,100)
    testPercentile = random.randint(0,100)
    for i in range (0, random.randint(5, 50)):
        testValues.append(testValue)
    testPercentile2(testValues,testPercentile)
    
        

Tree Creation time= 0.0 . Array Creation Time = 0.0 . Tree Better? False
Tree Percentile time= 0.0 . Array Percentile Time = 0.0010004043579101562 . Tree Better? True

Tree Creation time= 0.0009980201721191406 . Array Creation Time = 0.0 . Tree Better? False
Tree Percentile time= 0.0 . Array Percentile Time = 0.0 . Tree Better? False

Tree Creation time= 0.0 . Array Creation Time = 0.0 . Tree Better? False
Tree Percentile time= 0.0 . Array Percentile Time = 0.0 . Tree Better? False

Tree Creation time= 0.0 . Array Creation Time = 0.0 . Tree Better? False
Tree Percentile time= 0.0 . Array Percentile Time = 0.0009906291961669922 . Tree Better? True

Tree Creation time= 0.0 . Array Creation Time = 0.0010101795196533203 . Tree Better? True
Tree Percentile time= 0.0 . Array Percentile Time = 0.0009996891021728516 . Tree Better? True



In [40]:
#Test 6: Large repeated numbers

testValues = []

for i in range (0,100000):
    testValues.append(1)

testPercentile2(testValues, 33)

Tree Creation time= 0.20139765739440918 . Array Creation Time = 2.7131476402282715 . Tree Better? True
Tree Percentile time= 0.150071382522583 . Array Percentile Time = 15.028879880905151 . Tree Better? True



In [41]:
#Test7: 0 and 100 percentile

testValues = []

for i in range (0, 10000):
    testValues.append(random.randint(1,100))

testPercentile2(testValues,0)
testPercentile2(testValues,100)

Tree Creation time= 0.028616666793823242 . Array Creation Time = 0.1109311580657959 . Tree Better? True
Tree Percentile time= 0.022628068923950195 . Array Percentile Time = 0.3669261932373047 . Tree Better? True

Tree Creation time= 0.05187582969665527 . Array Creation Time = 0.13399219512939453 . Tree Better? True
Tree Percentile time= 0.05092501640319824 . Array Percentile Time = 0.3211536407470703 . Tree Better? True



In [42]:
#Test 8: single value

testPercentile2([0], 50)
testPercentile2([-3], 30)
testPercentile2([1000], 2)
testPercentile2([0], 0)

Tree Creation time= 0.0 . Array Creation Time = 0.0 . Tree Better? False
Tree Percentile time= 0.0 . Array Percentile Time = 0.0 . Tree Better? False

Tree Creation time= 0.0 . Array Creation Time = 0.0 . Tree Better? False
Tree Percentile time= 0.0 . Array Percentile Time = 0.0010001659393310547 . Tree Better? True

Tree Creation time= 0.0 . Array Creation Time = 0.0 . Tree Better? False
Tree Percentile time= 0.0 . Array Percentile Time = 0.0 . Tree Better? False

Tree Creation time= 0.0 . Array Creation Time = 0.0 . Tree Better? False
Tree Percentile time= 0.0 . Array Percentile Time = 0.0 . Tree Better? False



In [43]:
#Test 9: decimal percentiles

testValues = []

#for i in range (0, 10000):
    #testValues.append(random.randint(1,1000))
    
for i in range (0, 10):
    testValues.append(random.randint(1,100))

#testPercentile2(testValues,0.2)
#testPercentile2(testValues, 45)
testPercentile2(testValues, 20)


Tree Creation time= 0.0 . Array Creation Time = 0.0 . Tree Better? False
Tree Percentile time= 0.0 . Array Percentile Time = 0.0009853839874267578 . Tree Better? True



In [46]:
#Test 1: Google Nearest Rank Definition Test
#https://en.wikipedia.org/wiki/Percentile

testPercentile([15, 20, 35, 40, 50],5)
testPercentile([15, 20, 35, 40, 50],30)
testPercentile([15, 20, 35, 40, 50],40)
testPercentile([15, 20, 35, 40, 50],50)
testPercentile([15, 20, 35, 40, 50],100)

TypeError: 'int' object is not callable

In [30]:
#Test 1: Google Nearest Rank Definition Test
#https://en.wikipedia.org/wiki/Percentile

testPercentile([15, 20, 35, 40, 50],5)
testPercentile([15, 20, 35, 40, 50],30)
testPercentile([15, 20, 35, 40, 50],40)
testPercentile([15, 20, 35, 40, 50],50)
testPercentile([15, 20, 35, 40, 50],100)

Values =  [15, 20, 35, 40, 50]
Percentile =  5
Tree Creation time= 0.0 . Array Creation Time = 0.0 . Tree Better? True
Tree Percentile time= 0.0 . Array Percentile Time = 0.0010008811950683594 . Tree Better? True
Tree percentile =  15 . Array percentile =  15

Success!

Values =  [15, 20, 35, 40, 50]
Percentile =  30
Tree Creation time= 0.0 . Array Creation Time = 0.0 . Tree Better? True
Tree Percentile time= 0.0 . Array Percentile Time = 0.0 . Tree Better? True
Tree percentile =  20 . Array percentile =  20

Success!

Values =  [15, 20, 35, 40, 50]
Percentile =  40
Tree Creation time= 0.0 . Array Creation Time = 0.0 . Tree Better? True
Tree Percentile time= 0.0 . Array Percentile Time = 0.0 . Tree Better? True
Tree percentile =  20 . Array percentile =  20

Success!

Values =  [15, 20, 35, 40, 50]
Percentile =  50
Tree Creation time= 0.0 . Array Creation Time = 0.0 . Tree Better? True
Tree Percentile time= 0.0 . Array Percentile Time = 0.0010004043579101562 . Tree Better? True
Tree pe

In [47]:
#Test 2: Empty values test

testPercentile([],5)
testPercentile([],35)

TypeError: 'int' object is not callable